# Webscraping pour obtenir le lien du dataset

Le lien du dataset des discours de la BCE est mis à jour au début de chaque mois. Plutôt que d'aller le chercher "à la main", on peut utiliser le webscraping pour le modifier automatiquement dans nos requêtes (sous réserve qu'il n'y ait pas de modification dans la structure de la page du dataset).

In [1]:
import time

import urllib
import lxml.html
import requests
import bs4 #Beautiful Soup
import pandas as pd


### URL de la page de téléchargement

In [2]:
url_ecb_download="https://www.ecb.europa.eu/press/key/html/downloads.en.html"

In [3]:
requests.get(url_ecb_download).status_code
#Si le code retour est 200, il y a bien du contenu accessible sur la page.

200

### Liste de tous les liens de la page

In [4]:

def get_all_links(url):
    '''renvoie la liste de tous les liens présents sur la page correspondant à l'url'''
    list_links=[] #on initialise avec une liste vide
    STATUS_CODE=requests.get(url).status_code
    #Si le code retour est 200, il y a bien du contenu accessible sur la page.
    if STATUS_CODE!=200:
        print("ERREUR CODE", STATUS_CODE)
    else: #il y a bien du contenu accessible sur la page
        page = urllib.request.urlopen(url_ecb_download) # on se connecte à la page pour obtenir le code source
        dom =  lxml.html.fromstring(page.read()) #permet de "lire" le code source
        for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
            list_links.append(link)
        return list_links

In [5]:
#On obtient la liste de tous les liens présents sur la page
get_all_links(url_ecb_download)

['/home/html/index.en.html',
 '/home/html/index.en.html',
 'downloads.bg.html',
 'downloads.cs.html',
 'downloads.da.html',
 'downloads.de.html',
 'downloads.el.html',
 'downloads.en.html',
 'downloads.es.html',
 'downloads.et.html',
 'downloads.fi.html',
 'downloads.fr.html',
 'downloads.ga.html',
 'downloads.hr.html',
 'downloads.hu.html',
 'downloads.it.html',
 'downloads.lt.html',
 'downloads.lv.html',
 'downloads.mt.html',
 'downloads.nl.html',
 'downloads.pl.html',
 'downloads.pt.html',
 'downloads.ro.html',
 'downloads.sk.html',
 'downloads.sl.html',
 'downloads.sv.html',
 '/home/html/index.en.html',
 '/ecb/html/index.en.html',
 '/press/html/index.en.html',
 '/pub/html/index.en.html',
 '/stats/html/index.en.html',
 '/mopo/html/index.en.html',
 '/euro/html/index.en.html',
 '/paym/html/index.en.html',
 '/careers/html/index.en.html',
 'https://www.bankingsupervision.europa.eu/home/html/index.en.html',
 '#',
 '#',
 'javascript:window.print()',
 "javascript:window.open('https://twitt

### Lien du jeu de données 

In [6]:
def find_first_link_with_string(list_links, string):
    '''renvoie le premier lien de la liste contenant une certaine chaine de caractères'''
    for link in list_links:
        if string in link:
            return link

In [7]:
print(find_first_link_with_string(list_links=get_all_links(url_ecb_download),
                                  string="/press/key/shared/data/all_ECB_speeches.csv"))

/press/key/shared/data/all_ECB_speeches.csv?5f1650f87c081c670b20cc03e741c558


In [8]:
def get_dataset_url(url):
    '''renvoie la partie de l'url du dataset des discours de la BCE'''
    '''on utilise toutes les fonctions précédentes'''
    string="/press/key/shared/data/all_ECB_speeches.csv" #chaine de caractères identifiant le lien du dataset
    list_links=get_all_links(url) #on obtient la liste de tous les liens de la page
    url=find_first_link_with_string(list_links,string) #on cherche le lien du dataset grâce à la chaine de caractères
    return url

In [9]:
print(get_dataset_url(url_ecb_download))

/press/key/shared/data/all_ECB_speeches.csv?5f1650f87c081c670b20cc03e741c558


### Date de la dernière mise à jour des données

In [10]:
def last_update_dataset(url):
    '''renvoie la date de la dernière mise à jour du dataset'''
    STATUS_CODE=requests.get(url).status_code
    #Si le code retour est 200, il y a bien du contenu accessible sur la page.
    if STATUS_CODE!=200:
        print("ERREUR CODE", STATUS_CODE)
    else: #il y a bien du contenu accessible sur la page
        request_text = urllib.request.urlopen(url).read() #On se connecte à la page pour obtenir le code source
        page = bs4.BeautifulSoup(request_text, "lxml") 
        update_date=page.select_one("span[class*=lastUpdate]").text #on sélectionne le contenu de la classe lastUpdate
        return update_date

In [11]:
print(last_update_dataset(url=url_ecb_download))

last update: 01 January 2022


## Test de l'import du dataset

In [12]:
start=time.time()

url="https://www.ecb.europa.eu"+ get_dataset_url(url_ecb_download)

df=pd.read_csv(url,sep = "|",encoding='utf-8')

print("Temps pris :", time.time()-start) #23 secondes avec une connexion de 30 Mbps


Temps pris : 9.187734842300415


In [13]:
df

,date,speakers,title,subtitle,contents
0,2021-12-10,Fabio Panetta,The present and future of money in the digital...,"Lecture by Fabio Panetta, Member of the Execut...",SPEECH The present and future of money in ...
1,2021-12-08,Isabel Schnabel,Monetary policy and financial stability,"Speech by Isabel Schnabel, Member of the Execu...",SPEECH Monetary policy and financial stabi...
2,2021-12-08,Luis de Guindos,Macroprudential policy for non-bank financial ...,"Speech by Luis de Guindos, Vice-President of t...",SPEECH Macroprudential policy for non-bank...
3,2021-12-08,Christine Lagarde,Macroprudential policy in Europe – the future ...,"Welcome remarks by Christine Lagarde, Presiden...",SPEECH Macroprudential policy in Europe – ...
4,2021-11-29,Christine Lagarde,Dialogue in a changing world,"Lectio Magistralis by Christine Lagarde, Presi...",SPEECH Dialogue in a changing world Lect...
...,...,...,...,...,...
2548,1997-05-13,Alexandre Lamfalussy,The European Central Bank: independent and acc...,Keynote speech delivered by Alexandre Lamfalus...,The European Central Bank: independent and a...
2549,1997-04-30,Alexandre Lamfalussy,The operation of monetary policy in stage thre...,"Address by Alexandre Lamfalussy, President of ...",The operation of monetary policy in stage th...
2550,1997-04-22,Alexandre Lamfalussy,Convergence and the role of the European Centr...,"Remarks by Alexandre Lamfalussy, President of ...",Convergence and the role of the European Cen...
2551,1997-03-10,Alexandre Lamfalussy,Securing the benefits of EMU,"Address by Alexandre Lamfalussy, President of ...",Securing the benefits of EMU Address by Al...
